##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#need to use max length and padding to set explicit dimensions

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow_datasets as tfds
import tensorflow as tf

import time
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# import os
# import pprint

# if 'COLAB_TPU_ADDR' not in os.environ:
#   print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
# else:
#   tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
#   print ('TPU address is', tpu_address)

#   #with tf.Session(tpu_address) as session:
#   #  devices = session.list_devices()
    
#   #print('TPU devices:')
#   #pprint.pprint(devices)

# import tensorflow.logging

In [0]:
from sklearn.model_selection import train_test_split
from importlib import reload

BUFFER_SIZE = 20000
BATCH_SIZE = 32
d_model = 300

In [0]:
#!git clone https://github.com/ahathaway821/w266-gutenberg-quiz
#!mv /content/w266-gutenberg-quiz /content/w266_gutenberg_quiz
#!git -C ./w266_gutenberg_quiz reset --hard
#!git -C ./w266_gutenberg_quiz pull origin master

## Setup input pipeline

In [0]:
from w266_gutenberg_quiz.library.utility.squad_v3 import SquADDataSetV3

data_set = SquADDataSetV3(data_path="w266_gutenberg_quiz/data/SQuAD/train-v1.1.json")

test_size=.2
random_state=42
question_train, question_val, context_train, context_val, answer_train, answer_val = train_test_split(data_set.questions, 
                                                                                                      data_set.contexts, 
                                                                                                      data_set.answers, 
                                                                                                      test_size=test_size, 
                                                                                                      random_state=random_state)
train_examples = tf.data.Dataset.from_tensor_slices((question_train, context_train, answer_train))
val_examples = tf.data.Dataset.from_tensor_slices((question_val, context_val, answer_val))

In [0]:
print(len(data_set.answers))

In [0]:
filtered_question_train = []
filtered_question_val = []
filtered_context_train = []
filtered_context_val = []
filtered_answer_train = []
filtered_answer_val = []

max_answer_char_length = 50
max_question_char_length = 170
max_context_char_length = 2300
empty_a = 0
for i in range(0, len(question_train)):
  if len(answer_train[i]) < max_answer_char_length and len(question_train[i]) < max_question_char_length and len(context_train[i]) < max_context_char_length:
    if len(answer_train[i]) == 0:
      empty_a = empty_a + 1
      continue

    filtered_question_train.append(question_train[i])
    filtered_context_train.append(context_train[i])
    filtered_answer_train.append(answer_train[i])

for i in range(0, len(question_val)):
  if len(answer_val[i]) < max_answer_char_length and len(question_val[i]) < max_question_char_length and len(context_val[i]) < max_context_char_length:
    if len(answer_val[i]) == 0:
      continue
    filtered_question_val.append(question_val[i])
    filtered_context_val.append(context_val[i])
    filtered_answer_val.append(answer_val[i])

train_examples = tf.data.Dataset.from_tensor_slices((filtered_question_train, filtered_context_train, filtered_answer_train))
val_examples = tf.data.Dataset.from_tensor_slices((filtered_question_val, filtered_context_val, filtered_answer_val))

In [0]:
print(len(filtered_question_train))
print(len(filtered_question_val))

Create a custom subwords tokenizer from the training dataset. 

In [0]:
# Could combine all three to have common vocab?
# tokenizer_q = tfds.features.text.SubwordTextEncoder.build_from_corpus(
#   (question.numpy() for question, context, answer in train_examples), target_vocab_size=2**13)

# tokenizer_c = tfds.features.text.SubwordTextEncoder.build_from_corpus(
#   ((context + " " + answer).numpy() for question, context, answer in train_examples), target_vocab_size=2**13)

# tokenizer_a = tfds.features.text.SubwordTextEncoder.build_from_corpus(
#   ((context + " " + answer).numpy() for question, context, answer in train_examples), target_vocab_size=2**13)

# tokenizer_all = tfds.features.text.SubwordTextEncoder.build_from_corpus(
#    ((question + " " + context + " " + answer).numpy() for question, context, answer in train_examples), target_vocab_size=2**13)

# tokenizer_all = tfds.features.text.TokenTextEncoder(
#   ((question + " " + context + " " + answer).numpy() for question, context, answer in train_examples))


In [0]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for question, context, answer in train_examples:
  some_tokens = tokenizer.tokenize((question + " " + context + " " + answer).numpy())
  vocabulary_set.update(some_tokens)

tokenizer_all = tfds.features.text.TokenTextEncoder(vocabulary_set)
print(f'Vocab length: {len(vocabulary_set)}')

In [0]:
q_max_length = 0
a_max_length = 0
c_max_length = 0
for q_lang,c_lang,a_lang in train_examples:
  q_lang = tokenizer_all.encode(q_lang.numpy())
  if len(q_lang) > q_max_length:
    q_max_length = len(q_lang)

  c_lang = tokenizer_all.encode(c_lang.numpy())
  if len(c_lang) > c_max_length:
    c_max_length = len(c_lang)


  a_lang = tokenizer_all.encode(a_lang.numpy())
  if len(a_lang) > a_max_length:
    a_max_length = len(a_lang)

for q_lang,c_lang,a_lang in val_examples:
  q_lang = tokenizer_all.encode(q_lang.numpy())
  if len(q_lang) > q_max_length:
    q_max_length = len(q_lang)

  c_lang = tokenizer_all.encode(c_lang.numpy())
  if len(c_lang) > c_max_length:
    c_max_length = len(c_lang)


  a_lang = tokenizer_all.encode(a_lang.numpy())
  if len(a_lang) > a_max_length:
    a_max_length = len(a_lang)

# Add beginning and ending padding length
q_max_length = q_max_length + 2
c_max_length = c_max_length + 2
a_max_length = a_max_length + 2

print(f'Max Question Length: {q_max_length}')
print(f'Max Context Length: {c_max_length}')
print(f'Max Answer Length: {a_max_length}')

Add a start and end token to the input and target. 

In [0]:
def encode(q_lang, c_lang, a_lang):
  q_lang = [tokenizer_all.vocab_size] + tokenizer_all.encode(
      q_lang.numpy()) + [tokenizer_all.vocab_size+1]

  c_lang = [tokenizer_all.vocab_size] + tokenizer_all.encode(
      c_lang.numpy()) + [tokenizer_all.vocab_size+1]

  a_lang = [tokenizer_all.vocab_size] + tokenizer_all.encode(
      a_lang.numpy()) + [tokenizer_all.vocab_size+1]

  return q_lang, c_lang, a_lang

In [0]:
def tf_encode(q,c,a):
  return tf.py_function(encode, [q, c, a], [tf.int32, tf.int32, tf.int32])

In [0]:
train_dataset = train_examples.map(tf_encode)
#train_dataset = train_dataset.filter(filter_max_length)
val_dataset = val_examples.map(tf_encode)
#val_dataset = val_dataset.filter(filter_max_length)

# cache the dataset to memory to get a speedup while reading from it.
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(
    BATCH_SIZE,  padded_shapes=([q_max_length], [c_max_length], [a_max_length]))
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)


val_dataset = val_dataset.padded_batch(
    BATCH_SIZE,  padded_shapes=([q_max_length], [c_max_length], [a_max_length]))


In [0]:
for q,c,a in train_dataset.take(3):
  # print(q)
  # print(c)
  print(a)

# for q,c,a in val_dataset.take(2):
#   print(q)

In [0]:
q_batch, c_batch, a_batch = next(iter(val_dataset))
q_batch, c_batch, a_batch

## Positional encoding

Since this model doesn't contain any recurrence or convolution, positional encoding is added to give the model some information about the relative position of the words in the sentence. 

The positional encoding vector is added to the embedding vector. Embeddings represent a token in a d-dimensional space where tokens with similar meaning will be closer to each other. But the embeddings do not encode the relative position of words in a sentence. So after adding the positional encoding, words will be closer to each other based on the *similarity of their meaning and their position in the sentence*, in the d-dimensional space.

See the notebook on [positional encoding](https://github.com/tensorflow/examples/blob/master/community/en/position_encoding.ipynb) to learn more about it. The formula for calculating the positional encoding is as follows:

$$\Large{PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})} $$
$$\Large{PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})} $$

In [0]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [0]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
  pos_encoding = angle_rads[np.newaxis, ...]
    
  return tf.cast(pos_encoding, dtype=tf.float32)

## Masking

Mask all the pad tokens in the batch of sequence. It ensures that the model does not treat padding as the input. The mask indicates where pad value `0` is present: it outputs a `1` at those locations, and a `0` otherwise.

In [0]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

# def create_padding_mask(seq):
#   seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
#   # add extra dimensions to add the padding
#   # to the attention logits.
#   return seq[:, tf.newaxis, :]  # (batch_size, 1, seq_len)

The look-ahead mask is used to mask the future tokens in a sequence. In other words, the mask indicates which entries should not be used.

This means that to predict the third word, only the first and second word will be used. Similarly to predict the fourth word, only the first, second and the third word will be used and so on.

In [0]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

In [0]:
def f_mask(inputs, queries=None, keys=None, type=None):
    """Masks paddings on keys or queries to inputs
    inputs: 3d tensor. (N, T_q, T_k)
    queries: 3d tensor. (N, T_q, d)
    keys: 3d tensor. (N, T_k, d)
    """

    padding_num = -2 ** 32 + 1
    if type in ("k", "key", "keys"):
        # Generate masks
        masks = tf.sign(tf.reduce_sum(tf.abs(keys), axis=-1))  # (N, T_k)
        masks = tf.expand_dims(masks, 1) # (N, 1, T_k)
        masks = tf.tile(masks, [1, tf.shape(queries)[1], 1])  # (N, T_q, T_k)

        # Apply masks to inputs
        paddings = tf.ones_like(inputs) * padding_num

        outputs = tf.where(tf.equal(masks, 0), paddings, inputs)  # (N, T_q, T_k)
    elif type in ("q", "query", "queries"):
        # Generate masks
        masks = tf.sign(tf.reduce_sum(tf.abs(queries), axis=-1))  # (N, T_q)
        masks = tf.expand_dims(masks, -1)  # (N, T_q, 1)
        masks = tf.tile(masks, [1, 1, tf.shape(keys)[1]])  # (N, T_q, T_k)

        # Apply masks to inputs
        outputs = inputs*masks
    elif type in ("f", "future", "right"):
        diag_vals = tf.ones_like(inputs[0, :, :])  # (T_q, T_k)
        tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense()  # (T_q, T_k)
        masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(inputs)[0], 1, 1])  # (N, T_q, T_k)

        paddings = tf.ones_like(masks) * padding_num
        outputs = tf.where(tf.equal(masks, 0), paddings, inputs)
    else:
        print("Check if you entered type correctly!")

    return outputs

def create_masks(q, c, a):
  # Encoder padding mask
  q_enc_padding_mask = create_padding_mask(q)
  c_enc_padding_mask = create_padding_mask(c)

  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  q_dec_padding_mask = create_padding_mask(q)
  c_dec_padding_mask = create_padding_mask(c)

  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by 
  # the decoder.
  look_ahead_mask = create_look_ahead_mask(tf.shape(a)[1])
  dec_target_padding_mask = create_padding_mask(a)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

  
  return q_enc_padding_mask, c_enc_padding_mask, combined_mask, q_dec_padding_mask, c_dec_padding_mask

## Scaled dot product attention

<img src="https://www.tensorflow.org/images/tutorials/transformer/scaled_attention.png" width="500" alt="scaled_dot_product_attention">

The attention function used by the transformer takes three inputs: Q (query), K (key), V (value). The equation used to calculate the attention weights is:

$$\Large{Attention(Q, K, V) = softmax_k(\frac{QK^T}{\sqrt{d_k}}) V} $$

The dot-product attention is scaled by a factor of square root of the depth. This is done because for large values of depth, the dot product grows large in magnitude pushing the softmax function where it has small gradients resulting in a very hard softmax. 

For example, consider that `Q` and `K` have a mean of 0 and variance of 1. Their matrix multiplication will have a mean of 0 and variance of `dk`. Hence, *square root of `dk`* is used for scaling (and not any other number) because the matmul of `Q` and `K` should have a mean of 0 and variance of 1, and you get a gentler softmax.

The mask is multiplied with -1e9 (close to negative infinity). This is done because the mask is summed with the scaled matrix multiplication of Q and K and is applied immediately before a softmax. The goal is to zero out these cells, and large negative inputs to softmax are near zero in the output.

In [0]:
def scaled_dot_product_attention_v1(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask, extra=False):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention_v1(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

As the softmax normalization is done on K, its values decide the amount of importance given to Q.

The output represents the multiplication of the attention weights and the V (value) vector. This ensures that the words you want to focus on are kept as-is and the irrelevant words are flushed out.

## Multi-head attention

In [0]:
def scaled_dot_product_attention_v2(Q, K, V,
                                 num_heads,
                                 mask,
                                 causality=False, dropout_rate=0.,
                                 training=True,
                                 scope="scaled_dot_product_attention"):
    '''See 3.2.1.
    Q: Packed queries. 3d tensor. [N, T_q, d_k].
    K: Packed keys. 3d tensor. [N, T_k, d_k].
    V: Packed values. 3d tensor. [N, T_k, d_v].
    causality: If True, applies masking for future blinding
    dropout_rate: A floating point number of [0, 1].
    training: boolean for controlling droput
    scope: Optional scope for `variable_scope`.
    '''
    dk = Q.get_shape().as_list()[-1]
    #print('dk shape')
    #print(dk)

    #print('k shape')
    #print(K.shape)

    # dot product
    outputs = tf.matmul(Q, tf.transpose(K, [0, 2, 1]))  # (N, T_q, T_k)

    #print('outputs mha shape post dot product')
    #print(outputs.shape)

    # scale
    outputs /= dk ** 0.5

    # key masking, delete key 0
    #outputs = f_mask(outputs, Q, K, type="key")

    #print('outputs post mask')
    #print(outputs.shape)

    #print('outputs mha shape pre mask')
    #print(outputs.shape)

    # causality or future blinding masking
    if causality:
      outputs = f_mask(outputs, type="future")
    #if mask is not None:
    #  print('output shape')
    #  print(outputs.shape)
    #  print('mask shape')
    #  print(mask.shape)
    #  outputs += (mask * -1e9)  

    #print('outputs mha shape pre softmax')
    #print(outputs.shape)

    # softmax
    attn_dists = tf.nn.softmax(tf.reduce_sum(tf.split(outputs, num_heads, axis=0), axis=0))
    outputs = tf.nn.softmax(outputs)
    #print('outputs mha shape')
    #print(outputs.shape)
    attention = tf.transpose(outputs, [0, 2, 1])

    # query masking, delete query <pad>
    #outputs = f_mask(outputs, Q, K, type="query")

    # weighted sum (context vectors)
    outputs = tf.matmul(outputs, V)  # (N, T_q, d_v)

    return outputs, attn_dists

class MultiHeadAttention_v2(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention_v2, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask, causality=False):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    num_heads=4
    Q_ = tf.concat(tf.split(q, num_heads, axis=2), axis=0) # (h*N, T_q, d_model/h)
    K_ = tf.concat(tf.split(k, num_heads, axis=2), axis=0) # (h*N, T_k, d_model/h)
    V_ = tf.concat(tf.split(v, num_heads, axis=2), axis=0) # (h*N, T_k, d_model/h)

    # Attention
    dropout_rate=.1
    training=True
    outputs, attn_dists = scaled_dot_product_attention_v2(Q_, K_, V_, num_heads, mask, causality)

    # Restore shape
    outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2) # (N, T_q, d_model)
    # Residual connection
    #outputs = queries + outputs
          
    # Normalize
    #outputs = ln(outputs)
 
    return outputs, attn_dists

## Point wise feed forward network

Point wise feed forward network consists of two fully-connected layers with a ReLU activation in between.

In [0]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

## Elmo Embedding


In [0]:

#import hub
import tensorflow_hub as hub


class ElmoEmbeddingLayer(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    self.dimensions = 1024
    self.trainable = True
    super(ElmoEmbeddingLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable, name="{}_module".format(self.name))
    self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
    super(ElmoEmbeddingLayer, self).build(input_shape)
  def call(self, x, mask=None):
    result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                  as_dict=True,
                  signature='default',
                  )['default']
    return result
  def compute_mask(self, inputs, mask=None):
          return K.not_equal(inputs, '--PAD--')
  def compute_output_shape(self, input_shape):
          return (input_shape[0], self.dimensions)

In [0]:
elmo_embed_layer = hub.KerasLayer('https://tfhub.dev/google/elmo/3')

## Encoder and decoder

The transformer model follows the same general pattern as a standard [sequence to sequence with attention model](nmt_with_attention.ipynb). 

* The input sentence is passed through `N` encoder layers that generates an output for each word/token in the sequence.
* The decoder attends on the encoder's output and its own input (self-attention) to predict the next word. 

### Encoder layer

Each encoder layer consists of sublayers:

1.   Multi-head attention (with padding mask) 
2.    Point wise feed forward networks. 

Each of these sublayers has a residual connection around it followed by a layer normalization. Residual connections help in avoiding the vanishing gradient problem in deep networks.

The output of each sublayer is `LayerNorm(x + Sublayer(x))`. The normalization is done on the `d_model` (last) axis. There are N encoder layers in the transformer.

In [0]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):
    attn_output, att_weights = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2, att_weights

In [0]:
sample_encoder_layer = EncoderLayer(d_model, 4, 2048)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((64, 43, d_model)), False, None)

print(sample_encoder_layer_output[0].shape)  # (batch_size, input_seq_len, d_model)
print(sample_encoder_layer_output[1].shape)


### Decoder layer

Each decoder layer consists of sublayers:

1.   Masked multi-head attention (with look ahead mask and padding mask)
2.   Multi-head attention (with padding mask). V (value) and K (key) receive the *encoder output* as inputs. Q (query) receives the *output from the masked multi-head attention sublayer.*
3.   Point wise feed forward networks

Each of these sublayers has a residual connection around it followed by a layer normalization. The output of each sublayer is `LayerNorm(x + Sublayer(x))`. The normalization is done on the `d_model` (last) axis.

There are N decoder layers in the transformer.

As Q receives the output from decoder's first attention block, and K receives the encoder output, the attention weights represent the importance given to the decoder's input based on the encoder's output. In other words, the decoder predicts the next word by looking at the encoder output and self-attending to its own output. See the demonstration above in the scaled dot product attention section.

In [0]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)
    self.mha3 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)
 
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm4 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    self.dropout4 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, enc_output_q, enc_output_c, training, 
           look_ahead_mask, padding_mask_q, padding_mask_c):
    # enc_output.shape == (batch_size, input_seq_len, d_model)
    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask, True)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(enc_output_q, enc_output_q, out1, padding_mask_q)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    attn3, attn_weights_block3 = self.mha3(enc_output_c, enc_output_c, out2, padding_mask_c)  # (batch_size, target_seq_len, d_model)
    attn3 = self.dropout2(attn3, training=training)
    out3 = self.layernorm2(attn3 + out2)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out3)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out4 = self.layernorm3(ffn_output + out3)  # (batch_size, target_seq_len, d_model)

    return out4, attn_weights_block2, attn_weights_block3

In [0]:
sample_decoder_layer = DecoderLayer(d_model, 4, 2048)

sample_decoder_layer_output, _, _ = sample_decoder_layer(
    tf.random.uniform((64, 50, d_model)), sample_encoder_layer_output[0], sample_encoder_layer_output[1],
    False, None, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)

In [0]:
class PointerDecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, vocab_size, rate=0.1):
    super(PointerDecoderLayer, self).__init__()
    self.vocab_size = vocab_size
    self.mha1 = MultiHeadAttention_v2(d_model, num_heads)
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.ffn = point_wise_feed_forward_network(d_model, dff)
    #self.dropout1 = tf.keras.layers.Dropout(rate)
    self.gen_layer = tf.keras.layers.Dense(1, activation=tf.sigmoid, trainable=True, use_bias=False)


  def call(self, x, before_dec, enc_input, enc_output_q, enc_output_c, training=False):
    # enc_output.shape == (batch_size, input_seq_len, d_model)
    attn1, attn_weights_block1 = self.mha1(enc_output_q, enc_output_c, x, None, True)  # (batch_size, target_seq_len, d_model)
    #attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    ffn_output = self.ffn(out1)  # (batch_size, target_seq_len, d_model)
    #ffn_output = self.dropout1(ffn_output, training=training)
    x = self.layernorm1(ffn_output + out1)  # (batch_size, target_seq_len, d_model)

    # print('p x shape')
    # print(x.shape)

    # print('p before dec shape')
    # print(before_dec.shape)

    # print('p enc input shape')
    # print(enc_input.shape)

    # print('p att weight block')
    # print(attn_weights_block1.shape)

    weights = tf.transpose(embeddingLayer.weights[0]) # (d_model, vocab_size)
    logits = tf.einsum('ntd,dk->ntk', x, weights) # (N, T2, vocab_size)
    gens = self.gen_layer(tf.concat([before_dec, x, attn_weights_block1], axis=-1))
    logits = tf.nn.softmax(logits)
    logits = calc_final_dist(enc_input, gens, logits, attn_weights_block1, self.vocab_size)

    return logits

### Encoder

The `Encoder` consists of:
1.   Input Embedding
2.   Positional Encoding
3.   N encoder layers

The input is put through an embedding which is summed with the positional encoding. The output of this summation is the input to the encoder layers. The output of the encoder is the input to the decoder.

In [0]:

class Similarity(tf.keras.layers.Layer):

    def __init__(self, **kwargs):
        super(Similarity, self).__init__(**kwargs)

    def compute_similarity(self, repeated_context_vectors, repeated_query_vectors):
        element_wise_multiply = repeated_context_vectors * repeated_query_vectors
        concatenated_tensor = tf.keras.backend.concatenate(
            [repeated_context_vectors, repeated_query_vectors, element_wise_multiply], axis=-1)
        dot_product = tf.squeeze(tf.keras.backend.dot(concatenated_tensor, self.kernel), axis=-1)
        return tf.keras.activations.linear(dot_product + self.bias)

    def build(self, input_shape):
        word_vector_dim = input_shape[0][-1]
        weight_vector_dim = word_vector_dim * 3
        self.kernel = self.add_weight(name='similarity_weight',
                                      shape=(weight_vector_dim, 1),
                                      initializer='uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='similarity_bias',
                                    shape=(),
                                    initializer='ones',
                                    trainable=True)
        super(Similarity, self).build(input_shape)

    def call(self, inputs):
        context_vectors, query_vectors = inputs
        num_context_words = tf.shape(context_vectors)[1]
        num_query_words = tf.shape(query_vectors)[1]
        context_dim_repeat = tf.keras.backend.concatenate([[1, 1], [num_query_words], [1]], 0)
        query_dim_repeat = tf.keras.backend.concatenate([[1], [num_context_words], [1, 1]], 0)
        repeated_context_vectors = tf.tile(tf.expand_dims(context_vectors, axis=2), context_dim_repeat)
        repeated_query_vectors = tf.tile(tf.expand_dims(query_vectors, axis=1), query_dim_repeat)
        similarity_matrix = self.compute_similarity(repeated_context_vectors, repeated_query_vectors)
        return similarity_matrix

    def compute_output_shape(self, input_shape):
        batch_size = input_shape[0][0]
        num_context_words = input_shape[0][1]
        num_query_words = input_shape[1][1]
        return (batch_size, num_context_words, num_query_words)

    def get_config(self):
        config = super().get_config()
        return config


class C2QAttention(tf.keras.layers.Layer):

    def __init__(self, **kwargs):
        super(C2QAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        super(C2QAttention, self).build(input_shape)

    def call(self, inputs):
        similarity_matrix, encoded_question = inputs
        context_to_query_attention = tf.keras.layers.Softmax(axis=-1)(similarity_matrix)
        encoded_question = tf.expand_dims(encoded_question, axis=1)
        return tf.keras.backend.sum(tf.expand_dims(context_to_query_attention, axis=-1) * encoded_question, -2)

    def compute_output_shape(self, input_shape):
        similarity_matrix_shape, encoded_question_shape = input_shape
        return similarity_matrix_shape[:-1] + encoded_question_shape[-1:]

    def get_config(self):
        config = super().get_config()
        return config

class Q2CAttention(tf.keras.layers.Layer):

    def __init__(self, **kwargs):
        super(Q2CAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        super(Q2CAttention, self).build(input_shape)

    def call(self, inputs):
        similarity_matrix, encoded_context = inputs
        max_similarity = tf.keras.backend.max(similarity_matrix, axis=-1)
        # by default, axis = -1 in Softmax
        context_to_query_attention = tf.keras.layers.Softmax()(max_similarity)
        weighted_sum = tf.keras.backend.sum(tf.expand_dims(context_to_query_attention, axis=-1) * encoded_context, -2)
        expanded_weighted_sum = tf.expand_dims(weighted_sum, 1)
        num_of_repeatations = tf.shape(encoded_context)[1]
        return tf.tile(expanded_weighted_sum, [1, num_of_repeatations, 1])

    def compute_output_shape(self, input_shape):
        similarity_matrix_shape, encoded_context_shape = input_shape
        return similarity_matrix_shape[:-1] + encoded_context_shape[-1:]

    def get_config(self):
        config = super().get_config()
        return config

class MergedContext(tf.keras.layers.Layer):

    def __init__(self, **kwargs):
        super(MergedContext, self).__init__(**kwargs)

    def build(self, input_shape):
        super(MergedContext, self).build(input_shape)

    def call(self, inputs):
        encoded_context, context_to_query_attention, query_to_context_attention = inputs
        element_wise_multiply1 = encoded_context * context_to_query_attention
        element_wise_multiply2 = encoded_context * query_to_context_attention
        concatenated_tensor = tf.keras.backend.concatenate(
            [encoded_context, context_to_query_attention, element_wise_multiply1, element_wise_multiply2], axis=-1)
        return concatenated_tensor

    def compute_output_shape(self, input_shape):
        encoded_context_shape, _, _ = input_shape
        return encoded_context_shape[:-1] + (encoded_context_shape[-1] * 4, )

    def get_config(self):
        config = super().get_config()
        return config


In [0]:
embeddingLayer = tf.keras.layers.Embedding(tokenizer_all.vocab_size + 2, d_model)

In [0]:
class Encoder_QA(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, vocab_size,
               q_maximum_position_encoding, c_maximum_position_encoding, rate=0.1, num_q_model_layers=2, num_c_model_layers=2):
    super(Encoder_QA, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    self.num_q_model_layers = num_q_model_layers
    self.num_c_model_layers = num_c_model_layers
    
    #self.q_input = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
    #self.q_embedding = ElmoEmbeddingLayer()
    #self.q_embedding = tf.keras.layers.Embedding(vocab_size, d_model)

    self.q_pos_encoding = positional_encoding(q_maximum_position_encoding, 
                                            self.d_model)
    
    #self.c_input = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
    #self.q_embedding = ElmoEmbeddingLayer()
    #self.q_embedding.build(self.d_model)
    #self.q_embedding = elmo_embed_layer
    #self.c_embedding = tf.keras.layers.Embedding(vocab_size, d_model)
    self.c_pos_encoding = positional_encoding(c_maximum_position_encoding, 
                                            self.d_model)
    
    self.q_enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
    self.c_enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
    
    self.similarity = Similarity(name='similarity_layer')
        
    self.q_model_enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_q_model_layers)]
    self.c_model_enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_c_model_layers)]

  def call(self, q, c, training, q_mask, c_mask):
    

    q_seq_len = tf.shape(q)[1]
    c_seq_len = tf.shape(c)[1]
    
    #QUESTION
    # adding embedding and position encoding.
    #q = self.q_input(q)
    #q = self.q_embedding(q)  # (batch_size, input_seq_len, d_model)
    q = embeddingLayer(q)
    q *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    q += self.q_pos_encoding[:, :q_seq_len, :]

    q = self.dropout(q, training=training)
    
    for i in range(self.num_layers):
      q, q_concat_att = self.q_enc_layers[i](q, training, q_mask)

    #CONTEXT
    # adding embedding and position encoding.
    #c = self.c_input(c)
    c = embeddingLayer(c)  # (batch_size, input_seq_len, d_model)
    c *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    c += self.c_pos_encoding[:, :c_seq_len, :]

    c = self.dropout(c, training=training)
    
    for i in range(self.num_layers):
      c, c_concat_att  = self.c_enc_layers[i](c, training, c_mask)
    
    sim = self.similarity([c,q]) #(batch_size, context_max_length, query_max_length)

    context_to_query_attention = C2QAttention(name='context_to_query_attention')([
        sim, q]) #(batch_size, context_max_length, d_model)
    query_to_context_attention = Q2CAttention(name='query_to_context_attention')([
        sim, c]) #(batch_size, context_max_length, d_model)

    #merged_context = MergedContext(name='merged_context')(
    #    [c, context_to_query_attention, query_to_context_attention]) #(batch_size, context_max_length, d_model * 4)
    for i in range(self.num_q_model_layers):
      context_to_query_attention, q_model_concat_att = self.q_model_enc_layers[i](context_to_query_attention, training, None)

    for i in range(self.num_c_model_layers):
      query_to_context_attention, c_model_concat_att = self.c_model_enc_layers[i](query_to_context_attention, training, None)


    # print('Similarity Matrix Shape')
    # print(sim.shape)
    # print('Context to Query Att Shape')
    # print(context_to_query_attention.shape)
    # print('Query to Context Att Shape')
    # print(query_to_context_attention.shape)
    # print('Merged Context Shape')
    # print(merged_context.shape)
    #return q, c, q_concat_att, c_concat_att  # (batch_size, input_seq_len, d_model)
    return context_to_query_attention, query_to_context_attention, q_concat_att, c_concat_att

In [0]:
sample_encoder = Encoder_QA(num_layers=2, d_model=d_model, num_heads=4, 
                         dff=2048, vocab_size=8500,
                         q_maximum_position_encoding=10000, c_maximum_position_encoding=10000)
enc_temp_input = tf.random.uniform((64, 62), dtype=tf.int32, minval=0, maxval=200)
enc_temp_input_2 = tf.random.uniform((64, 100), dtype=tf.int32, minval=0, maxval=200)

sample_encoder_output = sample_encoder(enc_temp_input, enc_temp_input_2, training=False, q_mask=None, c_mask=None)

print (sample_encoder_output[0].shape)  # (batch_size, input_seq_len, d_model)
print (sample_encoder_output[1].shape)  # (batch_size, input_seq_len, d_model)

### Decoder

 The `Decoder` consists of:
1.   Output Embedding
2.   Positional Encoding
3.   N decoder layers

The target is put through an embedding which is summed with the positional encoding. The output of this summation is the input to the decoder layers. The output of the decoder is the input to the final linear layer.

In [0]:
def calc_final_dist(x, gens, vocab_dists, attn_dists, vocab_size):
    """Calculate the final distribution, for the pointer-generator model

    Args:
      x: encoder input which contain oov number
      gens: the generation, choose vocab from article or vocab
      vocab_dists: The vocabulary distributions. List length max_dec_steps of (batch_size, vsize) arrays.
                    The words are in the order they appear in the vocabulary file.
      attn_dists: The attention distributions. List length max_dec_steps of (batch_size, attn_len) arrays

    Returns:
      final_dists: The final distributions. List length max_dec_steps of (batch_size, extended_vsize) arrays.
    """

    # Multiply vocab dists by p_gen and attention dists by (1-p_gen)
    # print('vocab dist shape: List length max_dec_steps of (batch_size, vsize)')
    # print(vocab_dists.shape)

    # print('att dist shape: List length max_dec_steps of (batch_size, attn_len) arrays')
    # print(attn_dists.shape)

    # print('x shape')
    # print(x.shape)

    vocab_dists = gens * vocab_dists
    attn_dists = (1-gens) * attn_dists

    batch_size = tf.shape(attn_dists)[0]
    dec_t = tf.shape(attn_dists)[1]
    attn_len = tf.shape(attn_dists)[2]

    dec = tf.range(0, limit=dec_t) # [dec]
    dec = tf.expand_dims(dec, axis=-1) # [dec, 1]
    dec = tf.tile(dec, [1, attn_len]) # [dec, atten_len]
    # print('[dec, atten_len]')
    # print(dec.shape)
    dec = tf.expand_dims(dec, axis=0) # [1, dec, atten_len]
    # print('[1, dec, atten_len]')
    # print(dec.shape)
    dec = tf.tile(dec, [batch_size, 1, 1]) # [batch_size, dec, atten_len]

    x = tf.expand_dims(x, axis=1) # [batch_size, 1, atten_len]
    # print('new x shape: [batch_size, 1, atten_len]')
    # print(x.shape)
    x = tf.tile(x, [1, dec_t, 1]) # [batch_size, dec, atten_len]
    # #print('last x shape: [batch_size, dec, atten_len]')
    # #print(x.shape)
    x = tf.stack([dec, x], axis=3)

    attn_dists_projected = tf.map_fn(fn=lambda y: tf.scatter_nd(y[0], y[1], [dec_t, vocab_size]),
                                      elems=(x, attn_dists), dtype=tf.float32)

    final_dists = attn_dists_projected + vocab_dists

    return final_dists

In [0]:
class Decoder_QA(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder_QA, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    self.vocab_size = target_vocab_size
    self.num_heads=4
    self.depth = self.d_model // self.num_heads
    
    #self.dec_input = tf.keras.layers.Input(shape=(1,), dtype=tf.string)(self.d_model)
    #self.embedding = ElmoEmbeddingLayer()(dec_input, se.fd_model)

    #self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
    
    self.dec_layers= [DecoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)
    #self.add_att_layer = BahdanauAttention(d_model)
    #self.pointer_layer = PointerDecoderLayer(d_model, num_heads, dff, target_vocab_size)
    self.Wh = tf.keras.layers.Dense(1)
    self.Ws = tf.keras.layers.Dense(1)
    self.Wx = tf.keras.layers.Dense(1)
    self.V = tf.keras.layers.Dense(1)

  #enc_output[0] = question
  #enc_output[1] = context
  def call(self, x, enc_output, enc_input, training, 
           look_ahead_mask, q_padding_mask, c_padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}
    
    x = embeddingLayer(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
    
    before_x = x

    x = self.dropout(x, training=training)

    #print('enc output context shape')
    #print(enc_output[1].shape)

    for i in range(self.num_layers):
      # Query encoder ouput
      x, block1, block2 = self.dec_layers[i](x, enc_output[0], enc_output[1], training,
                                             look_ahead_mask, q_padding_mask, c_padding_mask)
      # print('q block1 shape')
      # print(block1.shape)
      
      # print('q block2 shape')
      # print(block2.shape)
      
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    # print('x shape')
    # print(x.shape)
    #print(enc_output[1].shape)
    #context_add = self.add_att_layer(x, enc_output[1])

    # print('enc input')
    # print(enc_input.shape)

    #x = self.pointer_layer(x, before_x, enc_input, enc_output[0], enc_output[1])
    #context vectors
    enc_out_shape = tf.shape(enc_output[0])
    context = tf.reshape(enc_output[1],(enc_out_shape[0], enc_out_shape[1], self.num_heads, self.depth) ) # shape : (batch_size, input_seq_len, num_heads, depth)
    context = tf.transpose(context, [0,2,1,3]) # (batch_size, num_heads, input_seq_len, depth)
    context = tf.expand_dims(context, axis=2)  # (batch_size, num_heads, 1, input_seq_len, depth)

    attn = tf.expand_dims(block2, axis=-1)  # (batch_size, num_heads, target_seq_len, input_seq_len, 1)

    context = context * attn # (batch_size, num_heads, target_seq_len, input_seq_len, depth)
    context = tf.reduce_sum(context, axis=3) # (batch_size, num_heads, target_seq_len, depth)
    context = tf.transpose(context, [0,2,1,3]) # (batch_size, target_seq_len, num_heads, depth)
    context = tf.reshape(context, (tf.shape(context)[0], tf.shape(context)[1], self.d_model)) # (batch_size, target_seq_len, d_model)

		# P_gens computing
    a = self.Wx(before_x)
    b = self.Ws(x)
    c = self.Wh(context)
    p_gens = tf.sigmoid(self.V(a + b + c))

    return x, attention_weights, p_gens

In [0]:
sample_decoder = Decoder_QA(num_layers=4, d_model=d_model, num_heads=4, 
                         dff=2048, target_vocab_size=tokenizer_all.vocab_size +2 ,
                         maximum_position_encoding=5000)

temp_input = tf.random.uniform((64, 26), dtype=tf.int32, minval=0, maxval=200)

output, attn, p_gens = sample_decoder(temp_input, 
                              enc_output=sample_encoder_output, 
                              enc_input=enc_temp_input_2, 
                              training=False,
                              look_ahead_mask=None, 
                              q_padding_mask=None,
                              c_padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape

# Currently trying to align encoder input shape

In [0]:
def _calc_final_dist( _enc_batch_extend_vocab, vocab_dists, attn_dists, p_gens, batch_oov_len, vocab_size, batch_size):
  """Calculate the final distribution, for the pointer-generator model
  Args:
  vocab_dists: The vocabulary distributions. List length max_dec_steps of (batch_size, vsize) arrays. The words are in the order they appear in the vocabulary file.
  attn_dists: The attention distributions. List length max_dec_steps of (batch_size, attn_len) arrays
  Returns:
  final_dists: The final distributions. List length max_dec_steps of (batch_size, extended_vsize) arrays.
  """
  # Multiply vocab dists by p_gen and attention dists by (1-p_gen)
  vocab_dists = [p_gen * dist for (p_gen,dist) in zip(p_gens, vocab_dists)]
  attn_dists = [(1-p_gen) * dist for (p_gen,dist) in zip(p_gens, attn_dists)]

  # Concatenate some zeros to each vocabulary dist, to hold the probabilities for in-article OOV words
  extended_vsize = vocab_size + batch_oov_len # the maximum (over the batch) size of the extended vocabulary
  extra_zeros = tf.zeros((batch_size, batch_oov_len ))
  vocab_dists_extended = [tf.concat(axis=1, values=[dist, extra_zeros]) for dist in vocab_dists] # list length max_dec_steps of shape (batch_size, extended_vsize)

  # Project the values in the attention distributions onto the appropriate entries in the final distributions
  # This means that if a_i = 0.1 and the ith encoder word is w, and w has index 500 in the vocabulary, then we add 0.1 onto the 500th entry of the final distribution
  # This is done for each decoder timestep.
  # This is fiddly; we use tf.scatter_nd to do the projection
  batch_nums = tf.range(0, limit=batch_size) # shape (batch_size)
  batch_nums = tf.expand_dims(batch_nums, 1) # shape (batch_size, 1)
  attn_len = tf.shape(_enc_batch_extend_vocab)[1] # number of states we attend over
  batch_nums = tf.tile(batch_nums, [1, attn_len]) # shape (batch_size, attn_len)
  indices = tf.stack( (batch_nums, _enc_batch_extend_vocab), axis=2) # shape (batch_size, enc_t, 2)
  shape = [batch_size, extended_vsize]
  attn_dists_projected = [tf.scatter_nd(indices, copy_dist, shape) for copy_dist in attn_dists] # list length max_dec_steps (batch_size, extended_vsize)

  # Add the vocab distributions and the copy distributions together to get the final distributions
  # final_dists is a list length max_dec_steps; each entry is a tensor shape (batch_size, extended_vsize) giving the final distribution for that decoder timestep
  # Note that for decoder timesteps and examples corresponding to a [PAD] token, this is junk - ignore.
  final_dists = [vocab_dist + copy_dist for (vocab_dist,copy_dist) in zip(vocab_dists_extended, attn_dists_projected)]

  return final_dists

## Create the Transformer

Transformer consists of the encoder, decoder and a final linear layer. The output of the decoder is the input to the linear layer and its output is returned.

In [0]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, vocab_size,
                  pe_input_q, pe_input_c, pe_target, rate=0.1):
    super(Transformer, self).__init__()
    self.vocab_size = vocab_size
    self.num_layers=4
    self.num_heads = num_heads
    self.encoder = Encoder_QA(num_layers, d_model, num_heads, dff, 
                           vocab_size, pe_input_q, pe_input_c, rate)

    self.decoder = Decoder_QA(num_layers, d_model, num_heads, dff, 
                           vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(vocab_size)
    
  def call(self, q, c, tar, training, q_enc_padding_mask, c_enc_padding_mask,
           look_ahead_mask, q_dec_padding_mask, c_dec_padding_mask):

    enc_output = self.encoder(q, c, training, q_enc_padding_mask, c_enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights, p_gens = self.decoder(
        tar, enc_output, c, training, look_ahead_mask, q_dec_padding_mask, c_dec_padding_mask)
    output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    output = tf.nn.softmax(output) # (batch_size, tar_seq_len, vocab_size)
    #output = tf.concat([output, tf.zeros((tf.shape(output)[0], tf.shape(output)[1], max_oov_len))], axis=-1) # (batch_size, targ_seq_len, vocab_size+max_oov_len)

    attn_dists = attention_weights['decoder_layer{}_block2'.format(self.num_layers)] # (batch_size,num_heads, targ_seq_len, inp_seq_len)
    attn_dists = tf.reduce_sum(attn_dists, axis=1)/self.num_heads # (batch_size, targ_seq_len, inp_seq_len)

    final_dists = _calc_final_dist( c, tf.unstack(output, axis=1) , tf.unstack(attn_dists, axis=1), tf.unstack(p_gens, axis=1), max_oov_len, self.vocab_size, BATCH_SIZE)
    final_output = tf.stack(final_dists, axis=1)

    return final_output, attention_weights

In [0]:
sample_transformer = Transformer(
    num_layers=4, d_model=d_model, num_heads=4, dff=2048, 
    vocab_size=tokenizer_all.vocab_size + 2, pe_input_q=10000, pe_input_c=10000, pe_target=6000)

temp_q = tf.random.uniform((64, 38), dtype=tf.int32, minval=0, maxval=200)
temp_c = tf.random.uniform((64, 38), dtype=tf.int32, minval=0, maxval=200)
temp_target = tf.random.uniform((64, 36), dtype=tf.int32, minval=0, maxval=200)

fn_out, _ = sample_transformer(temp_q, temp_c, temp_target, training=False, 
                               q_enc_padding_mask=None, 
                               c_enc_padding_mask=None,
                               look_ahead_mask=None,
                               q_dec_padding_mask=None,
                               c_dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

In [0]:
#from w266_gutenberg_quiz.library.models.seq2seq_att import Seq2SeqAtt
#qa = Seq2SeqAtt()
#qa.load_glove_model('w266_gutenberg_quiz/embeddings')
#qa.load_model(model_dir_path='w266_gutenberg_quiz/experiments/models')

In [0]:
import os

GLOVE_DIR ='/content/w266_gutenberg_quiz/embeddings'
d_model = 300

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

# prepare embedding matrix
from keras.initializers import Constant


num_words = len(tokenizer_all.tokens)
embedding_matrix = np.zeros((tokenizer_all.vocab_size + 2, d_model))
for i in range(0, num_words-1):
    word = tokenizer_all.tokens[i]
    if word[-1] == '_':
      word = word[0:-1]

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embeddingLayer = tf.keras.layers.Embedding(tokenizer_all.vocab_size + 2,
                            d_model,
                            embeddings_initializer=Constant(embedding_matrix),
                            trainable=False)

## Set hyperparameters

To keep this example small and relatively fast, the values for *num_layers, d_model, and dff* have been reduced. 

The values used in the base model of transformer were; *num_layers=6*, *d_model = 512*, *dff = 2048*. See the [paper](https://arxiv.org/abs/1706.03762) for all the other versions of the transformer.

Note: By changing the values below, you can get the model that achieved state of the art on many tasks.

In [0]:
num_layers = 4
#d_model = 256
dff = 512
num_heads = 4
vocab_size = tokenizer_all.vocab_size + 2
dropout_rate = 0.1

vocab_size

## Optimizer

Use the Adam optimizer with a custom learning rate scheduler according to the formula in the [paper](https://arxiv.org/abs/1706.03762).

$$\Large{lrate = d_{model}^{-0.5} * min(step{\_}num^{-0.5}, step{\_}num * warmup{\_}steps^{-1.5})}$$


In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [0]:
temp_learning_rate_schedule = CustomSchedule(d_model)

plt.plot(temp_learning_rate_schedule(tf.range(40000, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")

## Loss and metrics

Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [0]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_mean(loss_)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

## Training and checkpointing

In [0]:
#embeddingLayer = tf.keras.layers.Embedding(vocab_size, d_model)
# embeddingLayer = tf.keras.layers.Embedding(vocab_size),
#                             d_model,
#                             embeddings_initializer=Constant(embedding_matrix),
#                             trainable=False)
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          vocab_size,
                          pe_input_q=vocab_size, 
                          pe_input_c=vocab_size,
                          pe_target=vocab_size,
                          rate=dropout_rate)

loss = []
accuracy = []
e = []

checkpoint_path = "./checkpoints/train/v1"
#checkpoint_path = "/content/drive/My Drive/W266/FInalProject/transformer_v6_checkpoints/v`"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)
status = ckpt.restore(tf.train.latest_checkpoint(checkpoint_path))
print(status)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

Create the checkpoint path and the checkpoint manager. This will be used to save checkpoints every `n` epochs.

In [0]:
# # if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

In [0]:
EPOCHS = 50

In [0]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(BATCH_SIZE, q_max_length), dtype=tf.int32),
    tf.TensorSpec(shape=(BATCH_SIZE, c_max_length), dtype=tf.int32),
    tf.TensorSpec(shape=(BATCH_SIZE, a_max_length), dtype=tf.int32),
]

@tf.function(input_signature=train_step_signature)
def train_step(q, c, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  q_enc_padding_mask, c_enc_padding_mask, combined_mask, q_dec_padding_mask, c_dec_padding_mask = create_masks(q, c, tar_inp)
  with tf.GradientTape() as tape:
    predictions, _ = transformer(q, c, tar_inp, 
                                 True, 
                                 q_enc_padding_mask, 
                                 c_enc_padding_mask,
                                 combined_mask, 
                                 None,
                                 None)

  # with tf.GradientTape() as tape:
  #   predictions, _ = transformer(q, c, tar_inp, 
  #                                True, 
  #                                None, 
  #                                None,
  #                                None, 
  #                                None,
  #                                None)
    
    
    loss = loss_function(tar_real, predictions)


  gradients = tape.gradient(loss, transformer.trainable_variables)   
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

In [0]:

for epoch in range(EPOCHS):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  # question, context -> answer
  for (batch, (q, c, tar)) in enumerate(train_dataset):
    if q.shape[0] != BATCH_SIZE:
      continue
    train_step(q, c, tar)
    
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
      
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    
  print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))


  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))
  loss.append(train_loss.result())
  accuracy.append(train_accuracy.result())
  e.append(epoch)

In [0]:
import matplotlib

matplotlib.pyplot.plot(e, accuracy)
matplotlib.pyplot.xlabel('epochs')
matplotlib.pyplot.ylabel('accuracy')
matplotlib.pyplot.show()

matplotlib.pyplot.plot(e, loss)
matplotlib.pyplot.xlabel('epochs')
matplotlib.pyplot.ylabel('loss')
matplotlib.pyplot.show()

## Evaluate

In [0]:
def evaluate(q, c):
  q_start_token = [tokenizer_all.vocab_size]
  q_end_token = [tokenizer_all.vocab_size + 1]

  c_start_token = q_start_token
  c_end_token = q_end_token
  
  # inp sentence is portuguese, hence adding the start and end token
  q = q_start_token + tokenizer_all.encode(q) + q_end_token
  while len(q) < q_max_length:
    q.append(0)
  q_encoder_input = tf.expand_dims(q, 0)


  c = c_start_token + tokenizer_all.encode(c) + c_end_token
  while len(c) < c_max_length:
    c.append(0)
  c_encoder_input = tf.expand_dims(c, 0)
  
  # as the target is english, the first word to the transformer should be the
  # english start token.
  decoder_input = [tokenizer_all.vocab_size]
  output = tf.expand_dims(decoder_input, 0)
    
  for i in range(a_max_length + 200):
    q_enc_padding_mask, c_enc_padding_mask, combined_mask, q_dec_padding_mask, c_dec_padding_mask = create_masks(
        q_encoder_input, c_encoder_input, output)
  
    # print('q_enc padding mask')
    # print(q_enc_padding_mask.shape)
    # print('c enc padding mask')
    # print(c_enc_padding_mask.shape)
    # print('combined_mask')
    # print(combined_mask.shape)
    # print('q dec padding mask')
    # print(q_dec_padding_mask.shape)

    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(q_encoder_input, 
                                                 c_encoder_input,
                                                 output,
                                                 False,
                                                 q_enc_padding_mask,
                                                 c_enc_padding_mask,
                                                 combined_mask,
                                                 None,
                                                 None)
    
    # select the last word from the seq_len dimension
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    top_vals, top_indices = tf.nn.top_k(predictions[0][0], 5, sorted=True)
    #print(top_vals)
    #for i in top_indices:
    #  print(tokenizer_all.decode(i))

    #predicted_sentence = tokenizer_all.decode([i for i in top_indices 
    #                                        if i < tokenizer_all.vocab_size])
    #print(predicted_sentence)


    # return the result if the predicted_id is equal to the end token
    if predicted_id == tokenizer_all.vocab_size+1:
      #print('pred end')
      return tf.squeeze(output, axis=0), attention_weights
    
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0), attention_weights

In [0]:
def evaluate_beam_search(q, c, beam_size=2):
  q_start_token = [tokenizer_all.vocab_size]
  q_end_token = [tokenizer_all.vocab_size + 1]

  c_start_token = q_start_token
  c_end_token = q_end_token
  
  # inp sentence is portuguese, hence adding the start and end token
  q = q_start_token + tokenizer_all.encode(q) + q_end_token
  while len(q) < q_max_length:
    q.append(0)
  q_encoder_input = tf.expand_dims(q, 0)


  c = c_start_token + tokenizer_all.encode(c) + c_end_token
  while len(c) < c_max_length:
    c.append(0)
  c_encoder_input = tf.expand_dims(c, 0)
  
  # as the target is english, the first word to the transformer should be the
  # english start token.
  decoder_input = [tokenizer_all.vocab_size]
  output = tf.expand_dims(decoder_input, 0)
    
  for i in range(a_max_length + 200):
    q_enc_padding_mask, c_enc_padding_mask, combined_mask, q_dec_padding_mask, c_dec_padding_mask = create_masks(
        q_encoder_input, c_encoder_input, output)


  
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(q_encoder_input, 
                                                 c_encoder_input,
                                                 output,
                                                 False,
                                                 q_enc_padding_mask,
                                                 c_enc_padding_mask,
                                                 combined_mask,
                                                 None,
                                                 None)
    
    # select the last word from the seq_len dimension
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    top_vals, top_indices = tf.nn.top_k(predictions[0][0], 5, sorted=True)
    #print(top_vals)
    #for i in top_indices:
    #  print(tokenizer_all.decode(i))

    #predicted_sentence = tokenizer_all.decode([i for i in top_indices 
    #                                        if i < tokenizer_all.vocab_size])
    #print(predicted_sentence)


    # return the result if the predicted_id is equal to the end token
    if predicted_id == tokenizer_all.vocab_size+1:
      #print('pred end')
      return tf.squeeze(output, axis=0), attention_weights
    
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0), attention_weights

In [0]:
def translate(q, c, plot=''):
  result, attention_weights = evaluate(q, c)

  predicted_sentence = tokenizer_all.decode([i for i in result 
                                            if i < tokenizer_all.vocab_size])  

  print('Input Question: {}'.format(q))
  print('Input Context: {}'.format(c))
  print('Predicted translation: {}'.format(predicted_sentence))
  
  if plot:
     plot_attention_weights(attention_weights, q, c, plot)

In [0]:
context = """Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50."""
question =  "Which NFL team represented the AFC at Super Bowl 50?"

translate(question, context)
print ("Real translation: Denver Broncos")

In [0]:
tokenizer_all.vocab_size

In [0]:
# In training data
question = 'what people brought old english to britain ?'
context = 'old english nglisc , anglisc , englisc or anglo saxon is the earliest historical form of the english language , spoken in england and southern and eastern scotland in the early middle ages . it was brought to great britain by anglo saxon settlers probably in the mid th century , and the first old english literary works date from the mid th century . after the norman conquest of , english was replaced for a time as the language of the upper classes by anglo norman , a relative of french , and old english developed into the next historical form of english , known as middle english .'
translate(question, context)
print('Real answer: anglo saxon settlers')

In [0]:
context =  """The Apollo program, also known as Project Apollo, was the third United States human spaceflight program carried out by the National Aeronautics and Space Administration (NASA), which accomplished landing the first humans on the Moon from 1969 to 1972. First conceived during Dwight D. Eisenhower's administration as a three-man spacecraft to follow the one-man Project Mercury which put the first Americans in space, Apollo was later dedicated to President John F. Kennedy's national goal of "landing a man on the Moon and returning him safely to the Earth" by the end of the 1960s, which he proposed in a May 25, 1961, address to Congress. Project Mercury was followed by the two-man Project Gemini (1962–66). The first manned flight of Apollo was in 1968."""
question =  "What program was created to carry out these projects and missions?"

translate(question, context)
print ("Real translation: National Aeronautics and Space Administration (NASA)")

In [0]:
context =  """European Union law is a body of treaties and legislation, such as Regulations and Directives, which have direct effect or indirect effect on the laws of European Union member states. The three sources of European Union law are primary law, secondary law and supplementary law. The main sources of primary law are the Treaties establishing the European Union. Secondary sources include regulations and directives which are based on the Treaties. The legislature of the European Union is principally composed of the European Parliament and the Council of the European Union, which under the Treaties may establish secondary law to pursue the objective set out in the Treaties."""
question =  "What is European Union law?"

translate(question, context)
print ("Real translation: a body of treaties and legislation")

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# import shutil
# #!gsutil cp -r gs://checkpoints /drive/My Drive/W266/FInalProject/transformer_v2_checkpoints/
# shutil.copytree("/content/checkpoints", "drive/My Drive/W266/FInalProject/transformer_v5_checkpoints/v3") 


In [0]:
# !ls /content/checkpoints/train -al --file-type